In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install bs4

import sys

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

import requests
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
import pandas as pd
from collections import OrderedDict
from bs4.element import Comment
import urllib.request
import time
from selenium.webdriver.chrome.options import Options

import os

#how to run using multiple URLs
#seed_urls = ['https://www.wired.com/category/business',
#             'https://www.wired.com/category/backchannel',
#             'https://www.wired.com/category/gear']

#pull webpage and make a soup object
page = requests.get('https://www.bbc.com/')
soup = BeautifulSoup(page.content, 'html.parser')

#%% Get all sub-links on page
#empty lists for using below
articles = []
hold_heads = []
headlines = []
hold_sub = []
sub_articles = []

#scrape all URLs on main part of page
for body in soup.find_all(class_="content"): 
    for link in body.find_all("a"):
        articles.append(link.get('href'))

#pull out only the article URLs, leaving the advertisment
#and other non-pertinent links
for story in articles: 
    if "news" in story:
        hold_sub.append(story)

#remove duplicates by creating dictionary then 
#convert back to list; keeps order v set limitations
sub_articles = list(OrderedDict.fromkeys(hold_sub)) 

#grabbing all the headlines
for title in soup.find_all("h3"): 
    hold_heads.append(title.get_text())
headlines = list(OrderedDict.fromkeys(hold_heads))

#add prefix to articles for use in later function using concatenation
for i in range(len(sub_articles)): 
    sub_articles[i] = 'https://www.bbc.com' + sub_articles[i]


In [6]:
# instantiate a chrome options object so you can set the size and headless preference
#MOVE THIS DOWN TO FUNCTION
##chrome_options = Options()
##chrome_options.headless = True
##chrome_options.add_argument("--disable-gpu")
##chrome_options.add_argument("--window-size=1920x1080")

#chrome_driver = webdriver.Chrome(CHROMEDRIVER_PATH, chrome_options=options)
#chrome_options.add_argument("--headless")


##chrome_driver = r'C:\Users\ryanp\OneDrive\Documents\NIU\MST 691 Data Science Tools and Techniques\Code\chromedriver'

#driver = webdriver.Chrome(

In [7]:
def get_html_scroll(url):
    
    options = Options()
    options.headless = True
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920x1080")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    chrome_driver = r'C:\Users\ryanp\OneDrive\Documents\NIU\MST 691 Data Science Tools and Techniques\Code\chromedriver'
    browser = webdriver.Chrome(chrome_driver, options = options)
    browser.get(url)
    
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(4)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            time.sleep(4)
            match=True
    post_elms = browser.page_source
    return post_elms

In [4]:
#%% Scrape sub-links content
def build_dataset (sub_articles):
    article_data = []
    for url in sub_articles:
        data = get_html_scroll(url)        
        soup = BeautifulSoup(data, 'html.parser')
        
        news_articles = [{'article_headline': soup.find('h1').string,
                          'article_contents': soup.find_all('p')}]
        article_data.extend(news_articles)
        #time.sleep(10)
    df = pd.DataFrame(article_data)
    df = df[['article_headline', 'article_contents']]
    return df



In [8]:
#%% run program
news_df = build_dataset(sub_articles)
news_df.head(10)



WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=89.0.4389.114)
